Jupyter Notebook to run tl-rnn stepwise

In [1]:
    from util import *
    import pickle
    import pandas as pd
    import random
    from model import tlrnn
    from dataPrep import dataprep
    import pickle

Using TensorFlow backend.


finished


In [2]:
"""
define directory where you stored the data
"""
directory = "~/tl_rnn/data/"

In [3]:
"""
read training dataset and get rid of NA's
"""
df = pd.read_csv(directory + "sequential_data_train.csv", verbose=True)
df = df.fillna(0)

Tokenization took: 31.99 ms
Type conversion took: 12.88 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 9.27 ms
Type conversion took: 12.34 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 9.69 ms
Type conversion took: 13.67 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 10.46 ms
Type conversion took: 14.54 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 10.22 ms
Type conversion took: 14.28 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 10.26 ms
Type conversion took: 14.21 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 10.13 ms
Type conversion took: 14.29 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 9.68 ms
Type conversion took: 13.55 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 9.45 ms
Type conversion took: 15.44 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 12.98 ms
Type conversion took: 16.18 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 10.99 ms
Type conversion took: 15.1

In [4]:
"""
create dataprep object. in this object all hyperparameters for dataprocessing and sampling are stored
please double check all parameters in config.py before running it!!!
"""

dataobj = dataprep(dataframe=df, id_name="user_ID", sequence_id = "sequence_ID")

DATAFRAME COLUMNS  ['user_ID', 'sequence_ID', 'eventData_1', 'eventData_2', 'eventData_3']
SEQ_LEN_MAX 100
TOP 20 Categories of all Covariates ['cat_45', 'cat_73', 'cat_44', 'cat_26', 'cat_90', 'cat_213', 'cat_160', 'cat_69', 'cat_244', 'cat_232', 'cat_752', 'cat_133', 'cat_506', 'cat_672', 'cat_425', 'cat_403', 'cat_235', 'cat_390', 'cat_83', 'cat_547']
COVARIATES_ALL ['eventData_1', 'eventData_2', 'eventData_3']
SAMPLES PER USER 128
SEQUENCE LENGTH MAX 100


In [5]:
"""
        move data from original dataframe into lists of arrays,
        encode them properly: categories with a ranked based encoding
        one list entry per user array, with [timeframe index (e.g. nth week), seq_len(e.g. 1000), feature]
        time order conserved (t-1,t0,t+1)

        params: only members coming from dataprep init

        input:  columnwise dataframe read by dataframe() init

        return: list with each element being a user nested with another list of single array sequences:
                [user][array[timestep, features]]
"""

dataobj.preprocessor()

dataframe -> tensor ...
--------------------------------------------------
15/02/2022 19:21:03
0 of 9000 users 
--------------------------------------------------
15/02/2022 19:22:20
1000 of 9000 users 
--------------------------------------------------
15/02/2022 19:23:32
2000 of 9000 users 
--------------------------------------------------
15/02/2022 19:24:32
3000 of 9000 users 
--------------------------------------------------
15/02/2022 19:25:25
4000 of 9000 users 
--------------------------------------------------
15/02/2022 19:26:05
5000 of 9000 users 
--------------------------------------------------
15/02/2022 19:26:38
6000 of 9000 users 
--------------------------------------------------
15/02/2022 19:27:04
7000 of 9000 users 
--------------------------------------------------
15/02/2022 19:27:26
8000 of 9000 users 
tensor builder finished


In [6]:
"""
evaluate cardinalities of input values
this numbers will be taken into account to dimension the neural network model
"""

dataobj.build_cardinality()

{'eventData_1': 34808, 'eventData_2': 26, 'eventData_3': 33}


Before training with the training data, let's prepare the holdout (test) data-set to evaluate the resulting model

In [7]:
"""
read column-like TEST dataframe and get rid of NA's
"""

df_test = pd.read_csv(directory + "sequential_data_test.csv")
df_test = df_test.fillna(0)

In [8]:
"""
create dataprep object for test data
"""

dataobj_test = dataprep(dataframe=df_test, id_name="user_ID", sequence_id = "sequence_ID")

DATAFRAME COLUMNS  ['user_ID', 'sequence_ID', 'eventData_1', 'eventData_2', 'eventData_3']
SEQ_LEN_MAX 100
TOP 20 Categories of all Covariates ['cat_45', 'cat_73', 'cat_44', 'cat_90', 'cat_26', 'cat_69', 'cat_213', 'cat_232', 'cat_752', 'cat_160', 'cat_244', 'cat_251', 'cat_390', 'cat_672', 'cat_547', 'cat_324', 'cat_425', 'cat_506', 'cat_133', 'cat_402']
COVARIATES_ALL ['eventData_1', 'eventData_2', 'eventData_3']
SAMPLES PER USER 128
SEQUENCE LENGTH MAX 100


In [11]:
"""
VERY IMPORTANT!!

Overwrite the feature encodings (rank-based) with the ones from the training set
Otherwise training and test are encoded differently and the predicitve power gets bad
"""

dataobj_test.ranks = dataobj.ranks

In [12]:
"""
run preprocessor and bring datframe into propper list-like data structure for sampler
"""
dataobj_test.preprocessor()

dataframe -> tensor ...
--------------------------------------------------
15/02/2022 19:28:05
0 of 1000 users 
tensor builder finished


In [13]:
"""
Sample triplets from holdout (test) set to test re-identification score
Notice: In training mode, the triplets are sampled online during training,
for each epoch a new set of triplets
"""

dataobj_test.sampler(triplets_per_user=8, users_per_file="all")

triplets per user:  8
0 of 1232 users 
1000 of 1232 users 


Let's train the network:

In [14]:
"""
define cells and build tl-rnn keras model
put in dataobj, where also the pre-processed data-set
is included for "online" generator sampling
"""

my_tlrnn = tlrnn(cells=128, dataobj=dataobj)
my_tlrnn.build()

TypeError: __init__() missing 2 required positional arguments: 'inputs' and 'outputs'

SyntaxError: invalid syntax (<ipython-input-15-68601943eabe>, line 1)

In [14]:
dataobj.sampler(triplets_per_user=2, users_per_file="all")

triplets per user:  2
0 of 10000 users 
1000 of 10000 users 
2000 of 10000 users 
3000 of 10000 users 
4000 of 10000 users 
5000 of 10000 users 
6000 of 10000 users 
7000 of 10000 users 
8000 of 10000 users 
9000 of 10000 users 


In [15]:
my_tlrnn = tlrnn(cells=128, dataobj=dataobj)
my_tlrnn.build()

COVARIATES:  {'eventData_1': 35719, 'eventData_2': 25, 'eventData_3': 32}
SEQUENCE LEN MAX 100
CELLS 128
PATIENCE 1
BATCH SIZE 64
BATCHES PER USER 2
OPTIMIZER <keras.optimizers.Adam object at 0x7f3fcf675c88>


In [16]:
# train generator version

alpha=1.0

run_name = "test_insta_generator"

my_tlrnn.train_generator(run_name=run_name, alpha=alpha, test_set=dataobj.triplet_tensor)

split training and validation set
length training set:  9500
length validation set:  500
compile model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 1)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 100, 1)       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 100, 1)       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 100, 1)       0                                            
___

19000/19000 [==============================] - 487s 26ms/step - loss: 0.1716 - val_loss: 0.2360
score L1:  0.9426640625
Epoch 4/12
19000/19000 [==============================] - 484s 25ms/step - loss: 0.1328 - val_loss: 0.2261
score L1:  0.95375546875
Epoch 5/12
19000/19000 [==============================] - 486s 26ms/step - loss: 0.1079 - val_loss: 0.2135
score L1:  0.960203125
Epoch 6/12
19000/19000 [==============================] - 487s 26ms/step - loss: 0.0919 - val_loss: 0.2129
score L1:  0.9657203125
Epoch 7/12
19000/19000 [==============================] - 486s 26ms/step - loss: 0.0809 - val_loss: 0.2143
score L1:  0.9692328125
Epoch 00007: early stopping
training complete!


In [17]:

max_array = np.zeros((len(dataobj.cov_number)), dtype='int32')

for user_list in dataobj.data_list:
    for seq in user_list:
        for feature in dataobj.cov_number:
            if max(seq[:, feature]) > max_array[feature]:
                max_array[feature] = max(seq[:, feature])

max_array = max_array + 1


In [20]:
COVARIATES_ALL = ["eventData_1", "eventData_2", "eventData_3"]

COVARIATES_TO_TRANSLATE = ["eventData_1"]

In [21]:
res = []
i = 0
while (i < len(COVARIATES_ALL)):
    if (COVARIATES_TO_TRANSLATE.count(COVARIATES_ALL[i]) > 0):
        res.append(i)
    i += 1

In [23]:
list1 = [1,2,3,4]
list2 = [5,6,7,8]

In [29]:
3%1

0

In [15]:
for user_list in dataobj.data_list:
    for seq in user_list:
        print(seq)
        print("---------------")
        for feature in range(3):
            print("******************")
            print(seq[:, feature])
            print("******************")
            if max(seq[:, feature]) > max_array[feature]:
                max_array[feature] = max(seq[:, feature])


[[ 1804     9    11]
 [ 2962     9    11]
 [  164     9    11]
 [ 3472     9    11]
 [ 1356     9    11]
 [ 1078     9    11]
 [29333     9    11]
 [11025     9    11]
 [15511     9    11]
 [ 3317     9    11]
 [25435     9    11]
 [25445     9    11]
 [26191     9    11]
 [ 6018     9    11]
 [26405     9    11]]
---------------
******************
[ 1804  2962   164  3472  1356  1078 29333 11025 15511  3317 25435 25445
 26191  6018 26405]
******************


NameError: name 'max_array' is not defined